In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import solve_continuous_are

# System parameters (example values, replace with your own)
M = 0.3163     # Mass of cart
m1 = 0.0318    # Mass of pendulum 1
m2 = 0.0085    # Mass of pendulum 2
l1 = 0.316/2   # Length of pendulum 1
l2 = 0.079/2   # Length of pendulum 2
g = 9.81       # Gravity
I1 = 0.0085*(0.0098**2+0.0379^2)/12 + m1*((l1*2)**2)/3
I2 = 0.0085*(0.0098**2+0.0379^2)/12 + m2*((l2*2)**2)/3

# Damping
alpha = 12.2     # Carriage Slope
xdotss = 0.4852  # Terminal Velocity

a1 = 0.0185  # Damping coefficient pendulum 1
a2 = 0.012   # Damping coefficient pendulum 2

c = (M+m1+m2)*g*np.sin(alpha*np.pi/180)/xdotss  # Damping / Viscous Friction
c1 = 2*a1*I1 # Nms/rad    Viscous friction of pendulum 1 (rotational)
c2 = 2*a2*I2 # Nms/rad    Viscous friction of pendulum 2 (rotational)

# Linearized state-space matrices for double inverted pendulum (about upright)
A = np.zeros((6,6))
B = np.zeros((6,1))

# Fill in A and B for the linearized system (see reference for derivation)
# For brevity, here's a simplified version for small angles:
A[0,1] = 1
A[1,2] = (m1*g)/M
A[1,4] = (m2*g)/M
A[2,3] = 1
A[4,5] = 1

B[1,0] = 1/M

# LQR design
Q = np.diag([1000, 0, 100, 0, 100, 0])
R = np.array([[1]])

# Solve Riccati equation for LQR
P = solve_continuous_are(A, B, Q, R)
K = np.linalg.inv(R) @ B.T @ P

# Simulation parameters
T = np.linspace(0, 5, 2000)
dt = T[1] - T[0]
X = np.zeros((6, len(T)))
X[:,0] = [0, 0, 0.1, 0, 0.1, 0]  # Small initial angles

U = np.zeros(len(T))

def lqr_control(x):
    return -K @ x

# Simulate
for i in range(1, len(T)):
    u = lqr_control(X[:,i-1])
    U[i-1] = u
    x_dot = A @ X[:,i-1] + B.flatten() * u
    X[:,i] = X[:,i-1] + x_dot * dt

# Plot results
labels = ['Cart Position (m)', 'Cart Velocity (m/s)', 
          'Pendulum 1 Angle (rad)', 'Pendulum 1 Angular Velocity (rad/s)',
          'Pendulum 2 Angle (rad)', 'Pendulum 2 Angular Velocity (rad/s)']
plt.figure(figsize=(10, 12))
for idx in range(6):
    plt.subplot(6,1,idx+1)
    plt.plot(T, X[idx,:])
    plt.ylabel(labels[idx])
    if idx == 0:
        plt.title('Double Inverted Pendulum States Over Time')
    if idx == 5:
        plt.xlabel('Time (s)')
    plt.grid(True)
plt.tight_layout()
plt.show()